In [9]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_3d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, edge_preserving_smoothing_3d, image_smoothing_gaussian_3d
from skimage.morphology import remove_small_objects  

import pandas as pd

In [10]:
manifest_csv = pd.read_csv("//allen/aics/assay-dev/users/Sandi/aics-segmentation/lookup_table_demo/manifest.csv")

In [ ]:
#manifest_csv.head()

In [ ]:
#pd.DataFrame(manifest_csv.columns)

In [ ]:
tubulin_only = manifest_csv[(manifest_csv['structure_name'] == 'TUBA1B')]
tubulin_only.head()

In [ ]:
#for i in range(50):
    #print(tubulin_only['raw_fov_path'].iloc[i])

In [11]:
filename = "//allen/aics/assay-dev/users/Sandi/tubulin-segmentations/all-raw-imgs/20210312_N02_002-alignV2-Scene-24-P24-D02.tiff"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


(1, 2, 50, 1200, 1800)


In [15]:
N_CHANNELS = IMG.shape[1]
print(N_CHANNELS)
# MID_SLICE = np.int(0.5*IMG.shape[2])

# fig, ax = plt.subplots(1, N_CHANNELS, figsize=(18,16), dpi=72, facecolor='w', edgecolor='k')
# if N_CHANNELS==1:
#     ax.axis('off')
#     ax.imshow(IMG[0,0,0,MID_SLICE,:,:], cmap=plt.cm.gray)
# else:
#     for channel in range(N_CHANNELS):
#         ax[channel].axis('off')
#         ax[channel].imshow(IMG[0,0,channel,MID_SLICE,:,:], cmap=plt.cm.gray)

2


In [8]:
#####################
## PARAMETER ##
structure_channel = 0
#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()
view(single_fluorescent_view(struct_img0))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [84]:
################################
## PARAMETERS for this step ##
intensity_scaling_param = [0]
################################
# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with edge preserving smoothing 
structure_img_smooth = edge_preserving_smoothing_3d(struct_img)

#### for ZO1
# gaussian_smoothing_sigma = 1
# structure_img_smooth = image_smoothing_gaussian_3d(structure_img, sigma=gaussian_smoothing_sigma)

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [85]:
view(single_fluorescent_view(structure_img_smooth))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [86]:
from aicssegmentation.core.pre_processing_utils import suggest_normalization_param
suggest_normalization_param(struct_img0)

mean intensity of the stack: 126.18225860595703
the standard deviation of intensity of the stack: 95.80795288085938
0.9999 percentile of the stack intensity is: 1946.0
minimum intensity of the stack: 18.0
maximum intensity of the stack: 6328.0
suggested upper range is 19.0, which is 1946.5333633422852
suggested lower range is 1.0, which is 30.374305725097656
So, suggested parameter for normalization is [1.0, 19.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


In [ ]:
################################
## PARAMETERS for this step ##
f3_param = [[1, 0.01]]
################################

bw = filament_3d_wrapper(structure_img_smooth, f3_param)

In [ ]:
viewer_bw =view(segmentation_quick_view(bw))
viewer_bw

In [ ]:
# Option 1:
view(seg_fluo_side_by_side(struct_img,bw,roi=['ROI',viewer_bw.roi_slice()]))

# Option 2: 
#view(seg_fluo_side_by_side(struct_img,bw,roi=['M',[70,70,130,140]]))

In [ ]:
################################
## PARAMETERS for this step ##
minArea = 4
################################

seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)

In [ ]:
viewer_final = view(segmentation_quick_view(seg))
viewer_final

In [ ]:
# Option 1: 
view(seg_fluo_side_by_side(struct_img, seg, roi=['ROI',viewer_final.roi_slice()]))

# Option 2: 
# view(seg_fluo_side_by_side(struct_img, seg, roi=['M',[67,74, 188, 205]]))

In [ ]:
from tifffile import imsave

In [ ]:
imsave("tuba1b-d02s24.tiff", seg)